---
title: Crossover analysis
description: Using xOPR to automatically find radar crossovers
date: 2025-09-09
---

In this notebook, we demonstratate how to automatically find and analyze radar crossovers, both within and between campaigns.

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import xarray as xr
import geoviews as gv
import geoviews.feature as gf
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import shapely.geometry
import scipy.constants
import geopandas as gpd
from tqdm import tqdm

import xopr.opr_access
import xopr.geometry
import xopr.radar_util

import holoviews as hv
import hvplot.xarray
import hvplot.pandas
hvplot.extension('bokeh')

In [ ]:
# Useful projections
epsg_3031 = ccrs.Stereographic(central_latitude=-90, true_scale_latitude=-71)
latlng = ccrs.PlateCarree()
features = gf.ocean.options(scale='50m').opts(projection=epsg_3031) * gf.coastline.options(scale='50m').opts(projection=epsg_3031)

In [ ]:
# Establish an OPR session
# You'll probably want to set a cache directory if you're running this locally to speed
# up subsequent requests. You can do other things like customize the STAC API endpoint,
# but you shouldn't need to do that for most use cases.
opr = xopr.opr_access.OPRConnection(cache_dir="radar_cache")

# Or you can open a connection without a cache directory (for example, if you're parallelizing
# this on a cloud cluster without persistent storage).
#opr = xopr.OPRConnection()

In [ ]:
region = xopr.geometry.get_antarctic_regions(name='David', merge_regions=True, simplify_tolerance=100)

# Create a GeoViews object for the selected region
region_gv = gv.Polygons(region, crs=latlng).opts(
    color='green',
    line_color='black',
    fill_alpha=0.5,
    projection=epsg_3031,
)
features * region_gv

In [ ]:
stac_items = opr.query_frames(geometry=region)

# This feels sloppy, but there would be an easy fix if we just natively returned a GeoDataFrame from query_frames.
# It seems like it would generally be an improvement over a list and GeoPandas is already a dependency.

stac_items_df = gpd.GeoDataFrame(stac_items)
stac_items_df = stac_items_df.set_index('id')
stac_items_df = stac_items_df.set_geometry(stac_items_df['geometry'].apply(shapely.geometry.shape))
stac_items_df.crs = "EPSG:4326"
stac_items_df = stac_items_df.to_crs(epsg_3031)

print(f"Found {len(stac_items)} frames across {stac_items_df['collection'].nunique()} collections:")

stac_items_df.groupby('collection').size()

In [ ]:
flight_lines = stac_items_df.hvplot(by='collection')
(features * region_gv * flight_lines).opts(projection=epsg_3031, frame_width=600, aspect='equal')

In [ ]:
# Definitely room for improvement here, but the geopandas spatial join works pretty nicely for finding intersections between frames.
# Couple thoughts:
# 1. We might want to include a helper function for this
# 2. This will not find self-intersections within a single frame.

tmp_df = stac_items_df.reset_index()
tmp_df['geom'] = tmp_df.geometry
intersections = gpd.sjoin(tmp_df, tmp_df, how='inner', predicate='intersects', lsuffix='1', rsuffix='2')
intersections = intersections[intersections['id_1'] != intersections['id_2']]
intersections['intersection_geometry'] = intersections.apply(lambda row: row['geom_1'].intersection(row['geom_2']), axis=1)
intersections.set_geometry('intersection_geometry', inplace=True, crs=stac_items_df.crs)
intersections = intersections.drop_duplicates(subset=['intersection_geometry'])
intersections = intersections.explode(index_parts=True).reset_index(drop=True)
print(f"Found {len(intersections)} crossover points between flight lines.")
(features * region_gv * flight_lines * intersections.hvplot(label='Intersection Points', color='purple')).opts(frame_width=600, aspect='equal')

In [ ]:
# The result of this is a GeoDataFrame where every column from the intersecting frames is preserved, with suffixes _1 and _2 to distinguish them.
intersections.iloc[0]

In [ ]:
# This surfaced a lot of issues with layers...
# 1. It's annoying that you need to load a frame in order to get the layers for that flight. We should probably have a way to get layers directly from a STAC item.
# 1a. This work-around is OK for loading layer files, but it doesn't help with loading database layers. The databse layers include timing but not position information, so additional queries would be needed to get the position.
# 2. We should probably make both get_layers_files and get_layers_db return the same dataset structure.
# 3. Caching is a bit awkard here because get_layers_db will load the entire flight line.
# 4. The functionality to trim layers to the time range of the frame is broken in some cases because slow_time is sometimes returned as a float64 instead of a datetime64[ns]. We should fix this.

layer_cache = {}

def get_layers_stac_item(stac_item):
    ds_fake = xr.Dataset()
    ds_fake.attrs['season'] = stac_item['collection']
    ds_fake.attrs['segment'] = f"{stac_item['properties'].get('opr:date')}_{stac_item['properties'].get('opr:flight'):02d}"

    if (ds_fake.attrs['season'], ds_fake.attrs['segment']) in layer_cache:
        #print(f"Using cached layers for {ds_fake.attrs['season']} segment {ds_fake.attrs['segment']}...")
        return layer_cache[(ds_fake.attrs['season'], ds_fake.attrs['segment'])]

    #print(f"Loading layers for {ds_fake.attrs['season']} segment {ds_fake.attrs['segment']}...")

    layers = None
    layers = opr.get_layers_files(ds_fake)

    layer_cache[(ds_fake.attrs['season'], ds_fake.attrs['segment'])] = layers

    return layers

In [ ]:
layer_cache = {}

def get_basal_layer_wgs84(stac_item):
    flight_id = f"{stac_item['properties'].get('opr:date')}_{stac_item['properties'].get('opr:flight'):02d}"
    if (stac_item['collection'], flight_id) in layer_cache:
        #print(f"Using cached basal layer for {flight_id}")
        layers = layer_cache[(stac_item['collection'], flight_id)]
    else:
        try:
            layers = opr.get_layers_files(stac_item)
            if 2 not in layers:
                raise ValueError(f"No bed layer found in files for {flight_id}")
            # Don't cache file-based layers as they're loaded per-frame
        except Exception as e:
            layers = opr.get_layers_db(stac_item)
            layer_cache[(stac_item['collection'], flight_id)] = layers
    
    basal_layer = layers[2]
    surface_layer = layers[1]

    surface_wgs84 = layers[1]['elev'] - (layers[1]['twtt'] * (scipy.constants.c / 2))
    delta_twtt = basal_layer['twtt'] - surface_layer['twtt']
    basal_wgs84 = surface_wgs84 - (delta_twtt * ((scipy.constants.c / np.sqrt(3.15)) / 2))

    basal_layer['wgs84'] = basal_wgs84
    return basal_layer

In [ ]:
intersections['wgs84_1'] = np.nan
intersections['wgs84_2'] = np.nan
intersections['layer_pt_distance'] = np.nan

idx = 0
row = intersections.iloc[idx]

for idx, row in tqdm(intersections.iterrows(), total=len(intersections)):
    stac_item_1 = stac_items_df.loc[row['id_1']].to_dict()
    stac_item_2 = stac_items_df.loc[row['id_2']].to_dict()

    try:
        bed_1 = get_basal_layer_wgs84(stac_item_1).rename({'lat': 'Latitude', 'lon': 'Longitude'})
        bed_2 = get_basal_layer_wgs84(stac_item_2).rename({'lat': 'Latitude', 'lon': 'Longitude'})

        bed_1 = xopr.geometry.project_dataset(bed_1, "EPSG:3031")
        bed_2 = xopr.geometry.project_dataset(bed_2, "EPSG:3031")

        x, y = row.intersection_geometry.coords[0]

        dist_1 = np.sqrt((bed_1['x'] - x)**2 + (bed_1['y'] - y)**2)
        dist_2 = np.sqrt((bed_2['x'] - x)**2 + (bed_2['y'] - y)**2)

        min_idx_1 = dist_1.argmin().item()
        min_idx_2 = dist_2.argmin().item()

        dist_between_pts = np.sqrt((bed_1['x'][min_idx_1] - bed_2['x'][min_idx_2])**2 + (bed_1['y'][min_idx_1] - bed_2['y'][min_idx_2])**2)

        elev_1 = bed_1['wgs84'][min_idx_1].item()
        elev_2 = bed_2['wgs84'][min_idx_2].item()

        intersections.at[idx, 'wgs84_1'] = elev_1
        intersections.at[idx, 'wgs84_2'] = elev_2
        intersections.at[idx, 'layer_pt_distance'] = dist_between_pts
    except Exception as e:
        print(f"Error processing intersection {idx} between {row['id_1']} and {row['id_2']}: {repr(e)}")
        #raise e
        #traceback.print_exc()

intersections['elev_diff'] = intersections['wgs84_1'] - intersections['wgs84_2']
# Set elev_diff to NaN where layer_pt_distance is large
intersections.loc[intersections['layer_pt_distance'] > 100, 'elev_diff'] = np.nan

In [ ]:
intersections_success = intersections.dropna().reset_index(drop=True)
intersections_success['idx'] = intersections_success.index
hover_tooltips = [
    ("Index", "@idx"),
    ("Collection 1", "@collection_1"),
    ("Collection 2", "@collection_2"),
    ("Difference", "@elev_diff{0.00} m"),
]

vlim = intersections_success['elev_diff'].abs().quantile(0.95)

hv_int = intersections_success.hvplot(color='elev_diff', cmap='coolwarm_r', hover_cols=['idx', 'collection_1', 'collection_2', 'elev_diff'], hover_tooltips=hover_tooltips, clim=(-vlim, vlim))
hv_int = hv_int.opts(scalebar=True)
(features * region_gv * flight_lines * hv_int).opts(frame_width=600, aspect='equal', active_tools=['pan', 'wheel_zoom'])

In [ ]:
selected_idx = 53

In [ ]:
# Get intersection details
intersect = intersections_success.loc[selected_idx]
stac_1 = stac_items_df.loc[intersect['id_1']].to_dict()
stac_2 = stac_items_df.loc[intersect['id_2']].to_dict()

# Load frames
frame_1 = opr.load_frame(stac_1)
frame_1 = xopr.radar_util.add_along_track_coordinate(frame_1)
frame_1 = xopr.radar_util.interpolate_to_vertical_grid(frame_1, vertical_coordinate='wgs84')
frame_2 = opr.load_frame(stac_2)
frame_2 = xopr.radar_util.add_along_track_coordinate(frame_2)
frame_2 = xopr.radar_util.interpolate_to_vertical_grid(frame_2, vertical_coordinate='wgs84')

# Project to EPSG:3031 and find closest points to intersection
x_int, y_int = intersect.intersection_geometry.coords[0]
frame_1_proj = xopr.geometry.project_dataset(frame_1, "EPSG:3031")
frame_2_proj = xopr.geometry.project_dataset(frame_2, "EPSG:3031")

# Find indices closest to intersection
dist_1 = np.sqrt((frame_1_proj['x'] - x_int)**2 + (frame_1_proj['y'] - y_int)**2)
dist_2 = np.sqrt((frame_2_proj['x'] - x_int)**2 + (frame_2_proj['y'] - y_int)**2)
idx_1 = dist_1.argmin().item()
idx_2 = dist_2.argmin().item()

print(f"Frame 1: {intersect['id_1']} from {intersect['collection_1']}")
print(f"Frame 2: {intersect['id_2']} from {intersect['collection_2']}")
print(f"Intersection at index {idx_1} (frame 1) and {idx_2} (frame 2)")
print(f"Bed elevation difference: {intersect['elev_diff']:.2f} m")

In [ ]:
def add_layers_to_frame(frame):
    speed_in_ice = scipy.constants.c / np.sqrt(3.15)  # Approximate speed of light in ice (m/s)

    layers = opr.get_layers_files(frame)
    if len(layers) == 0:
        layers = opr.get_layers_db(frame)
    
    has_surface = False

    for layer_idx in layers:
        frame[f'layer_{layer_idx}_twtt'] = layers[layer_idx]['twtt'].interp(coords={'slow_time': frame['slow_time']})

        if layer_idx == 1:
            has_surface = True
            frame[f'layer_{layer_idx}_range'] = frame['layer_1_twtt'] * (scipy.constants.c / 2)
            frame[f'layer_{layer_idx}_elevation'] = frame['Elevation'] - frame[f'layer_1_range']
        elif has_surface:
            twtt_from_surface = frame[f'layer_{layer_idx}_twtt'] - frame['layer_1_twtt']
            frame[f'layer_{layer_idx}_range'] = (twtt_from_surface * (speed_in_ice / 2)) + frame['layer_1_range']
            frame[f'layer_{layer_idx}_elevation'] = frame['Elevation'] - frame[f'layer_{layer_idx}_range']

    return frame

# Load layers for both frames
frame_1 = add_layers_to_frame(frame_1)
frame_2 = add_layers_to_frame(frame_2)

In [ ]:
# Plot radargrams in elevation coordinates with layers
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 8))

# Frame 1 radargram in elevation
pwr_1_elev = 10*np.log10(np.abs(frame_1.Data))
pwr_1_elev.plot.imshow(x='along_track', y='wgs84', cmap='gray', ax=ax1)
ax1.axvline(frame_1.along_track[idx_1].values, color='red', linestyle='--', linewidth=2, label='Crossover')

# Plot layers using elevation data
frame_1['layer_1_elevation'].plot(ax=ax1, x='along_track', linestyle=':', color='tab:blue', linewidth=2, label='Surface')
frame_1['layer_2_elevation'].plot(ax=ax1, x='along_track', linestyle=':', color='tab:orange', linewidth=2, label='Bed')

ax1.set_title(f"{intersect['collection_1']} - {intersect['id_1']} (Elevation view)")
ax1.set_ylabel('Elevation (m)')
ax1.legend()

# Frame 2 radargram in elevation
pwr_2_elev = 10*np.log10(np.abs(frame_2.Data))
pwr_2_elev.plot.imshow(x='along_track', y='wgs84', cmap='gray', ax=ax2)
ax2.axvline(frame_2.along_track[idx_2].values, color='red', linestyle='--', linewidth=2, label='Crossover')

# Plot layers using elevation data
frame_2['layer_1_elevation'].plot(ax=ax2, x='along_track', linestyle=':', color='tab:blue', linewidth=2, label='Surface')
frame_2['layer_2_elevation'].plot(ax=ax2, x='along_track', linestyle=':', color='tab:orange', linewidth=2, label='Bed')

ax2.set_title(f"{intersect['collection_2']} - {intersect['id_2']} (Elevation view)")
ax2.set_xlabel('Along track distance (m)')
ax2.set_ylabel('Elevation (m)')
ax2.legend()

plt.suptitle(f"Radargrams in elevation coordinates - Bed elev diff: {intersect['elev_diff']:.2f} m", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Zoomed elevation plots around crossover
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 8))

window_size = 200  # Number of traces on each side of intersection
elev_window = 300  # Elevation window in meters around bed

# Frame 1 zoomed
idx_start_1 = max(0, idx_1 - window_size)
idx_end_1 = min(len(frame_1.slow_time), idx_1 + window_size)
bed_elev_1 = frame_1['layer_2_elevation'].isel(slow_time=idx_1).values
elev_min_1 = bed_elev_1 - elev_window/2
elev_max_1 = bed_elev_1 + elev_window/2

pwr_1_zoom = frame_1.Data.isel(slow_time=slice(idx_start_1, idx_end_1))
pwr_1_zoom_db = 10*np.log10(np.abs(pwr_1_zoom))
pwr_1_zoom_db.plot.imshow(x='along_track', y='wgs84', cmap='gray', ax=ax1)
ax1.axvline(frame_1.along_track[idx_1].values, color='red', linestyle='--', linewidth=2, label='Crossover')

# Plot layers
layer_1_zoom = frame_1['layer_1_elevation'].isel(slow_time=slice(idx_start_1, idx_end_1))
layer_1_zoom.plot.scatter(ax=ax1, x='along_track', linestyle=':', color='tab:blue', linewidth=2, label='Surface', s=4)
layer_2_zoom = frame_1['layer_2_elevation'].isel(slow_time=slice(idx_start_1, idx_end_1))
layer_2_zoom.plot.scatter(ax=ax1, x='along_track', linestyle=':', color='tab:orange', linewidth=2, label='Bed', s=4)

ax1.set_ylim(elev_min_1, elev_max_1)
ax1.set_title(f"{intersect['collection_1']} - Zoomed (Bed: {intersect['wgs84_1']:.1f} m)")
ax1.set_ylabel('Elevation (m)')
ax1.legend()

# Frame 2 zoomed
idx_start_2 = max(0, idx_2 - window_size)
idx_end_2 = min(len(frame_2.slow_time), idx_2 + window_size)
bed_elev_2 = frame_2['layer_2_elevation'].isel(slow_time=idx_2).values
elev_min_2 = bed_elev_2 - elev_window/2
elev_max_2 = bed_elev_2 + elev_window/2

pwr_2_zoom = frame_2.Data.isel(slow_time=slice(idx_start_2, idx_end_2))
pwr_2_zoom_db = 10*np.log10(np.abs(pwr_2_zoom))
pwr_2_zoom_db.plot.imshow(x='along_track', y='wgs84', cmap='gray', ax=ax2)
ax2.axvline(frame_2.along_track[idx_2].values, color='red', linestyle='--', linewidth=2, label='Crossover')

# Plot layers
layer_1_zoom = frame_2['layer_1_elevation'].isel(slow_time=slice(idx_start_2, idx_end_2))
layer_1_zoom.plot.scatter(ax=ax2, x='along_track', linestyle=':', color='tab:blue', linewidth=2, label='Surface', s=4)
layer_2_zoom = frame_2['layer_2_elevation'].isel(slow_time=slice(idx_start_2, idx_end_2))
layer_2_zoom.plot.scatter(ax=ax2, x='along_track', linestyle=':', color='tab:orange', linewidth=2, label='Bed', s=4)

ax2.set_ylim(elev_min_2, elev_max_2)
ax2.set_title(f"{intersect['collection_2']} - Zoomed (Bed: {intersect['wgs84_2']:.1f} m)")
ax2.set_xlabel('Along track distance (m)')
ax2.set_ylabel('Elevation (m)')
ax2.legend()

plt.suptitle(f"Elevation crossover comparison - Bed difference: {intersect['elev_diff']:.2f} m", fontsize=14)
plt.tight_layout()
plt.show()